# Import

In [1]:
import pandas as pd
# from fugue_sql import fsql
#Single model classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
#Ensemble classifications models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,confusion_matrix, classification_report, roc_auc_score,
f1_score, recall_score, precision_score, SCORERS)
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier

In [2]:
Modelname = 'RandomForestClassifier_model.sav'


# Functions

In [3]:
def baseline_models(data=[], verbose=False):
    #List of models to be used
    models=[DecisionTreeClassifier(),LogisticRegression(),
            RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=101),
            GradientBoostingClassifier(learning_rate=0.1,random_state=1,n_estimators=100,min_samples_split=5,min_samples_leaf=1,subsample=0.8,max_depth=3),
            AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200,random_state=1,learning_rate=1)
]
    #Create training and testing data sets depending on wheather or not they have been generated previously.
    if len(data)== 2:
        X_train, X_test, y_train, y_test = train_test_split(data[0],data[1],random_state=123)
    else:
        X_train, X_test, y_train, y_test = data[0],data[1],data[2],data[3]
    #Instantiate lists to store each of the models results
    accuracy = []
    f1 = []
    auc = []
    recall = []
    precision = []
    #Run thorugh each of the models to get their performance metrics
    for model in models:
        clf = model
        clf.fit(X_train, y_train)
        test_preds = clf.predict(X_test)
        f1.append(f1_score(y_test, test_preds,average='macro'))
        accuracy.append(accuracy_score(y_test, test_preds))
        # auc.append(roc_auc_score(y_test, test_preds,multi_class="ovr" ))
        recall.append(recall_score(y_test, test_preds,average='macro'))
        precision.append(precision_score(y_test, test_preds,average='macro'))
        #Print the model and its report
        if verbose:
            print('Classification Model: ',model,'\n')
            print(classification_report(y_test, test_preds),'\n')
    #store results in dataframe
    results = pd.DataFrame([f1, accuracy, precision,recall],
                      index= ['f1','accuracy','precision','recall',],
                           columns=['DecisionTree','LogisticRegression','RandomForest','Gradient Boosting','AdaBoostClassifier'])
    #Change orientation of the dataframe
    return results.transpose()

# ML

In [4]:
path='/home/o/oananbeh/notebook/Experment2/src/TabTransformer-multiclass_discharge/'

In [5]:
# #connect the SOMTE_DataSet_died with SOMTE_DataSet_discharge
# died_DF=pd.read_csv(path+'SOMTE_DataSet_prepare_association_died_0.5_0.7.csv')
# died_DF['OUTCOME']=0
# discharge_DF=pd.read_csv(path+'SOMTE_DataSet_prepare_association_discharge_0.5_0.7.csv')
# discharge_DF['OUTCOME']=1
# frames = [died_DF, discharge_DF]
# Filal_SMOTE_DataSet=pd.concat(frames)

In [6]:
# Filal_SMOTE_DataSet.shape

In [7]:
# Filal_SMOTE_DataSet.to_csv(path+'Filal_SMOTE_DataSet.csv',index=False)

In [8]:
Filal_SMOTE_DataSet=pd.read_csv(path+'SMOT_discharge_DataSet.csv')

In [9]:
target = Filal_SMOTE_DataSet.OUTCOME
print(f"Original class counts: {Counter(target)}")

Original class counts: Counter({'discharge=<1': 85, 'discharge=<2': 85, 'discharge=<3': 85, 'discharge=<4': 85, 'discharge>4': 85})


In [10]:
ohc_feats_nonNumericalFeatures = pd.get_dummies(Filal_SMOTE_DataSet)


In [11]:
# ohc_feats_nonNumericalFeatures.to_csv(path+'ohc_feats_Features')

In [12]:
# load the dataset
dataset =  Filal_SMOTE_DataSet
# retrieve the array of data
data = dataset.values
# separate into input and output columns
X = data[:, :-1].astype(str)
y = data[:, -1].astype(str)
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print('X_train',X_train.shape,'X_test',X_test.shape)
 
print(f"Original class counts y_train: {Counter(y_train)}")
print(f"Original class counts y_test: {Counter(y_test)}")
# one-hot encode input variables
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoder.fit(X_train)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)


X_train (297, 91) X_test (128, 91)
Original class counts y_train: Counter({'discharge=<1': 62, 'discharge=<2': 61, 'discharge>4': 60, 'discharge=<3': 58, 'discharge=<4': 56})
Original class counts y_test: Counter({'discharge=<4': 29, 'discharge=<3': 27, 'discharge>4': 25, 'discharge=<2': 24, 'discharge=<1': 23})


In [13]:
results = baseline_models(data=[X_train, X_test, y_train, y_test])
results.sort_values('f1',ascending=False)

/home/o/oananbeh/.conda/envs/notebook-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/o/oananbeh/.conda/envs/notebook-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,f1,accuracy,precision,recall
RandomForest,0.814971,0.812500,0.873810,0.808687
Gradient Boosting,0.791391,0.789062,0.822086,0.786589
DecisionTree,0.779624,0.781250,0.807052,0.775227
LogisticRegression,0.770435,0.773438,0.779781,0.768486
AdaBoostClassifier,0.554899,0.578125,0.652834,0.581820


# RandomForestClassifier

In [14]:
df_train = Filal_SMOTE_DataSet.drop('OUTCOME', axis=1)
train_labels = Filal_SMOTE_DataSet['OUTCOME']

In [15]:
df_train.shape[1]

91

In [16]:
encoders = []
features = []
for i in range(91):
    encoder_i = OneHotEncoder(handle_unknown='ignore')
    i_column = df_train.columns[i]
    encoder_i.fit(df_train[i_column].values.reshape(-1, 1))
    encoded_feature = encoder_i.transform(df_train[i_column].values.reshape(-1, 1))
    encoded_feature = encoded_feature.toarray()
    features.append(encoded_feature)
    encoders.append(encoder_i)

def get_oh(df):
    features_test = []
    for i in range(91):
        encoder_i = encoders[i]
        i_column = Filal_SMOTE_DataSet.columns[i]
        encoded_test = encoder_i.transform(df[i_column].values.reshape(-1, 1))
        encoded_test = encoded_test.toarray()
        features_test.append(encoded_test)
    return  np.concatenate(features_test, axis=1)

train_features = np.concatenate(features, axis=1)


In [17]:
train_features.shape, train_labels.values.shape

((425, 3364), (425,))

In [18]:
# separate into input and output columns
X = train_features
y = train_labels
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
# one-hot encode input variables
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

In [19]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
# save the model
pickle.dump(clf, open(path+Modelname, 'wb'))